# Operar con datos en Pandas

Una de las piezas esenciales de NumPy es la capacidad de realizar operaciones rápidas elemento a elemento, tanto con aritmética básica (suma, resta, multiplicación, etc.) como con operaciones más sofisticadas (funciones trigonométricas, funciones exponenciales y logarítmicas, etc.).
**Pandas hereda gran parte de esta funcionalidad de NumPy, y las ufuncs** que introdujimos en [Computation on NumPy Arrays: Universal Functions](3_Computation-on-arrays-ufuncs.ipynb) son clave para ello.

Pandas incluye un par de giros útiles, sin embargo: para operaciones unarias como negación y funciones trigonométricas, estas ufuncs *preservarán las etiquetas de índice y columna* en la salida, y para operaciones binarias como **adición y multiplicación, Pandas *alineará automáticamente los índices* al pasar los objetos a la ufunc.**
Esto significa que mantener el contexto de los datos y combinar datos de diferentes fuentes -ambas tareas potencialmente propensas a errores con arrays NumPy sin procesar- se convierten en tareas esencialmente infalibles con Pandas.
Además, veremos que hay operaciones bien definidas entre estructuras unidimensionales ``Series`` y estructuras bidimensionales ``DataFrame``.

## Ufuncs: Índice Preservación

Como Pandas está diseñado para trabajar con NumPy, cualquier ufunc de NumPy funcionará con los objetos ``Series`` y ``DataFrame`` de Pandas.
Empecemos definiendo una ``Series`` y un ``DataFrame`` simples para demostrarlo:

In [1]:
import pandas as pd
import numpy as np

>> #### **RandomState** = Seed o Semilla de Numpy.

Tiene el mismo efecto, genera como un código al número/os aleatorios y lo devuelve con el índice.

In [3]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4)) # 4 números, del 0 al 10
ser

0    6
1    3
2    7
3    4
dtype: int32

> DIFERENCIA DE RANDOM EN SERIES Y DataFrame:

- En **series** puede guardarse con RandomState. 
- En los **DataFrame**, no se guarda, siempre va a cambiar la secuencia de números aleatorios. 

In [6]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),  # Números del 0 al 10, en DataFrame de 3 filas y 4 columnas
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,3,8,2
1,4,2,6,4
2,8,6,1,3


Si aplicamos una ufunc NumPy sobre cualquiera de estos objetos, el resultado será otro objeto Pandas *con los índices conservados:*

- **exp**= exponencial > pero multiplica el número por el **número E** ***(Euler's number)*** = 2.718281 > Que es lo contrario de logaritmo natural
- **ser** = variable de la serie de arriba

In [7]:
ser

0    6
1    3
2    7
3    4
dtype: int32

- Sería NÚMERO E = 2.718.. elevado a la exponencial de cada número de la serie 
- Ejemplo: 2.71 ^ 6 --- 2.71 ^ 3, etc

In [8]:
np.exp(ser) 

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

O, para un cálculo un poco más complejo:
- **sin** = seno 
- **df** = DataFrame. En el código de abajo del DataFrame se aplica a cada elemento por separado. 
  
Sería = seno del (DataFrame x número pi / 4)

In [9]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


Cualquiera de las ufuncs discutidas en [Computation on NumPy Arrays: Universal Functions](3_Computation-on-arrays-ufuncs.ipynb) puede ser usada de forma similar.

## UFuncs: Index Alignment

Para operaciones binarias sobre dos objetos ``Series`` o ``DataFrame``, Pandas alineará los índices en el proceso de realización de la operación.
Esto es muy conveniente cuando se trabaja con datos incompletos, como veremos en algunos de los ejemplos que siguen.

### Alineación del índice en la serie

Como ejemplo, supongamos que estamos combinando dos fuentes de datos diferentes, y encontramos sólo los tres primeros estados de EE.UU. por *área* y los tres primeros estados de EE.UU. por *población*:

In [14]:
area = pd.Series({'Alaska': 1723337,
                  'Texas': 695662,
                  'California': 423967},
                 name='area')

population = pd.Series({'California': 38332521,
                        'Texas': 26448193,
                        'New York': 19651127},
                       name='population')

In [12]:
print(area)

Alaska        1723337
Texas          695662
California     423967
dtype: int64


In [13]:
print(population)

California    38332521
Texas         26448193
New York      19651127
Name: population, dtype: int64


Veamos qué ocurre cuando los dividimos para calcular la densidad de población:

In [9]:
type(population / area)

pandas.core.series.Series

- Al ser NaN, porque no hay otro número por el cual dividirlo, sale NaN > Que NO es lo mismo que dividir por 0, que ahí sí saldría error.

In [10]:
s_poblacion_area = population / area
print(s_poblacion_area)

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64


De ALASKA y NUEVA YORK devuelve un NaN, porque no existe en la otra serie, o sea, no está en los dos, sino que una serie es distinta a la otra. 

La matriz resultante contiene la *unión* de los índices de las dos matrices de entrada, que podría determinarse utilizando la aritmética de conjuntos estándar de Python sobre estos índices:

In [13]:
area.index

Index(['Alaska', 'Texas', 'California'], dtype='object')

In [14]:
population.index

Index(['California', 'Texas', 'New York'], dtype='object')

In [16]:
area.index.union(population.index)

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

Cualquier elemento para el que uno u otro no tenga una entrada se marca con ``NaN``, o "Not a Number", que es como Pandas marca los datos que faltan (ver más discusión sobre datos que faltan en [Handling Missing Data](4_Missing-Values.ipynb)).
Esta coincidencia de índices se implementa de esta forma para cualquiera de las expresiones aritméticas incorporadas en Python; cualquier valor que falte se rellena con NaN por defecto:

In [17]:
A = pd.Series([2, 4, 6], 
              index=["andalucia", "aragon", "madrid"])
print(A)

B = pd.Series([1, 3, 5], 
              index=["aragon", "madrid", "asturias"])
print(B)
A + B

andalucia    2
aragon       4
madrid       6
dtype: int64
aragon      1
madrid      3
asturias    5
dtype: int64


andalucia    NaN
aragon       5.0
asturias     NaN
madrid       9.0
dtype: float64

- CREO EL DataFrame CON LAS 2 SERIES ANTERIORES:

In [18]:
df_AB = pd.DataFrame({'n_alumnos': A, 'n_profesores': B})
df_AB

,n_alumnos,n_profesores
andalucia,2.0,NaN
aragon,4.0,1.0
asturias,NaN,5.0
madrid,6.0,3.0


- Creo una NUEVA COLUMNA con la suma de las 2 existentes:

In [19]:
# df_AB = pd.DataFrame({'n_alumnos': A, 'n_profesores': B})
df_AB['n_personas'] = df_AB['n_alumnos'] + df_AB['n_profesores']
df_AB

,n_alumnos,n_profesores,n_personas
andalucia,2.0,NaN,NaN
aragon,4.0,1.0,5.0
asturias,NaN,5.0,NaN
madrid,6.0,3.0,9.0


- Creo OTRA NUEVA COLUMNA con la multiplicación de una columna por 10

In [20]:
df_AB['n_alumnos_10'] = df_AB['n_alumnos'] * 10
df_AB

,n_alumnos,n_profesores,n_personas,n_alumnos_10
andalucia,2.0,NaN,NaN,20.0
aragon,4.0,1.0,5.0,40.0
asturias,NaN,5.0,NaN,NaN
madrid,6.0,3.0,9.0,60.0


Si el uso de valores NaN no es el comportamiento deseado, el valor de relleno puede modificarse utilizando métodos de objeto apropiados en lugar de los operadores.
Por ejemplo, llamar a ``A.add(B)`` es equivalente a llamar a ``A + B``, pero permite la especificación explícita opcional del valor de relleno para cualquier elemento de ``A`` o ``B`` que pueda faltar:

In [21]:
B + A

andalucia    NaN
aragon       5.0
asturias     NaN
madrid       9.0
dtype: float64

In [22]:
print(A)
print(B)

andalucia    2
aragon       4
madrid       6
dtype: int64
aragon      1
madrid      3
asturias    5
dtype: int64


>> **.fill_value**

Rellena el valor de los campos donde no haya, por ejemplo de los NaN. Funciona como un try except ya que si tiene valor, no lo modifica, y si no tiene, lo rellena con el parámetro que le paso. 

In [25]:
A.add(B, fill_value=0) #Devuelve la suma de los valores, pero sin NaN, ya que valen 0
                       # porque fill_value=0, es como un try/except por si no tiene valor. 

andalucia    2.0
aragon       5.0
asturias     5.0
madrid       9.0
dtype: float64

### Alineación de índices en DataFrame

Un tipo similar de alineación tiene lugar para *ambos* columnas e índices cuando se realizan operaciones en ``DataFrame``s:

In [15]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

,A,B
0,13,17
1,8,1


In [18]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

,B,A,C
0,5,9,3
1,5,1,9
2,1,9,3


>> #### SUMA DE DOS DATAFRAME:

1. **SUMA NORMAL**
2. **SUMA MEDIANTE .add()**
3. **SUMA + .fillna()** 

Los datos que no coinciden según el índice porque no existe la columna o la fila, coloca NaN

> **1. SUMA NORMAL**

In [29]:
A + B

,A,B,C
0,22.0,22.0,NaN
1,9.0,6.0,NaN
2,NaN,NaN,NaN


**CAMBIO DE ETIQUETAS DE COLUMNAS EN UN DATAFRAME Y ORDEN DE LAS MISMAS EN SUMA DE DF**
Cuando sumo 2 DataFrame, las columnas se ordenan por orden natural según nombre/etiqueta de las mismas. 
Si quiero cambiar el orden de las columnas en el nuevo DataFrame, puedo elegir el orden de la siguiente manera:

In [ ]:
c = A + B
c[['B','C','A']]

Observe que los índices se alinean correctamente independientemente de su orden en los dos objetos, y que los índices del resultado están ordenados.
Como en el caso de ``Series``, podemos utilizar el método aritmético del objeto asociado y pasarle cualquier ``valor_de_relleno`` que queramos utilizar en lugar de las entradas que falten.
Aquí rellenaremos con la media de todos los valores de ``A`` (calculada apilando primero las filas de ``A``):

> **2. SUMA MEDIANTE .add()** 

Suma a los valores que NO tienen coincidencia, la media de los valores de A

In [24]:
fill = A.values.mean()
A.add(B, fill_value=fill)


,A,B,C
0,22.00,22.00,12.75
1,9.00,6.00,18.75
2,18.75,10.75,12.75


> **3. SUMA + .fillna()** 

Suma los valores existentes y coincidentes según índice, y los vacíos (NaN) los rellena con un valor que le paso entre parentesis. Ejemplo: 0

In [25]:
A.add(B).fillna(0)

,A,B,C
0,22.0,22.0,0.0
1,9.0,6.0,0.0
2,0.0,0.0,0.0


#### **La siguiente tabla lista los operadores de Python y sus métodos equivalentes en los objetos de Pandas:**

| Python Operador | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Ufuncs: Operaciones entre DataFrame y Series

Cuando se realizan operaciones entre un ``DataFrame`` y una ``Series``, la alineación de índices y columnas se mantiene de forma similar.
Las operaciones entre un ``DataFrame`` y una ``Series`` son similares a las operaciones entre un array NumPy bidimensional y unidimensional.
Consideremos una operación común, donde encontramos la diferencia de un array bidimensional y una de sus filas:

In [ ]:
A = rng.randint(10, size=(3, 4))
A

In [ ]:
A - A[0]

Según las reglas de difusión de NumPy (véase [Computación en matrices: difusión](5_Computation-on-arrays-broadcasting.ipynb)), la resta entre una matriz bidimensional y una de sus filas se aplica fila a fila.

En Pandas, la convención opera de forma similar por defecto:

In [ ]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

Si, por el contrario, desea operar por columnas, puede utilizar los métodos de objeto mencionados anteriormente, especificando la palabra clave ``axis``:

In [ ]:
df.subtract(df['R'], axis=0)

Tenga en cuenta que estas operaciones ``DataFrame``/``Series``, al igual que las operaciones comentadas anteriormente, alinearán automáticamente los índices entre los dos elementos:

In [ ]:
df

In [ ]:
halfrow = df.iloc[0, ::2]
halfrow

In [ ]:
df - halfrow

Esta preservación y alineación de índices y columnas significa que las operaciones sobre datos en Pandas siempre mantendrán el contexto de los datos, lo que previene los tipos de errores tontos que podrían surgir al trabajar con datos heterogéneos y/o desalineados en arrays NumPy sin procesar.